# Final Project
Data Cleaning

In [87]:
import pandas as pd

data = pd.read_csv('urinalysis_tests.csv')

#I didn't know what to do with these columns but we should implement these at some point
data = data.drop(columns=['WBC', 'RBC'], axis=1)

data['Diagnosis'] = data['Diagnosis'].map({'POSITIVE': 1, 'NEGATIVE': 0})
data['Gender'] = data['Gender'].map({'MALE': 1, 'FEMALE': 0})

#One-Hot Encoding
#Idk if one-hot encoding everything is maybe bad cause of dimensionality
data = pd.get_dummies(data, columns=['Color', 'Transparency', 'Glucose', 'Protein', 'Epithelial Cells', 'Mucous Threads', 'Amorphous Urates', 'Bacteria'])

features = data.drop('Diagnosis', axis=1)
diagnosis = data['Diagnosis']

Decision Tree

In [88]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

X_train, X_test, Y_train, Y_test = train_test_split(features, diagnosis, test_size=0.2, random_state=5)

tree = DecisionTreeClassifier(random_state=5)
tree.fit(X_train, Y_train)
predictions = tree.predict(X_test)

accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

precision = precision_score(Y_test, predictions)
print("Precision:", precision)

recall = recall_score(Y_test, predictions)
print("Recall:", recall)

Accuracy: 0.9097222222222222
Precision: 0.058823529411764705
Recall: 0.09090909090909091


# Clustering

Scale our Data

In [97]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(features)

K-Nearest Neighbors Clustering

In [106]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

knn = KNeighborsClassifier(5)

knn.fit(data_scaled, diagnosis)
knn_Clusters = knn.predict(data_scaled)

knn_Silhouette = silhouette_score(data_scaled, knn_Clusters)

print("Silhouette Score:", knn_Silhouette)

Silhouette Score: 0.20545125303413


DBSCAN Clustering

In [91]:
from sklearn.cluster import DBSCAN

eps = 0.5
minPts = 5
dbscan = DBSCAN(eps=eps, min_samples=minPts)


dbscan-clusters = dbscan.fit_predict(X_scaled)
knn-silhouette = silhouette_score(X_scaled, knn-clusters)

print("Silhouette Score:", dbscan-silhouette)